#Integrantes
Micaela Mangen, Morena Lopez Obejero

#Consigna
1. Plantear un problema a resolver con una red neuronal de una capa de entrada y una capa de salida. Las salidas no deben ser excluyentes, es decir, pueden pertenecer a mas de una clase. Algunos ejemplos son: A partir de caracteristicas de una vivienda, determinar si la vivienda es de algun estilo (moderna, agradable, tradicional). Pueden generar ustedes un pequeño dataset para utilizar o buscar en Kaggle.
2. Modificar la estructura de la red neuronal para que admita
n
 neuronas de entrada y
m
 neuronas de salida.

Ademas deben indicar los elementos de la teoría que vimos en clase que identifiquen en el código y explicarlos (Estructura de la Red, Función de activación, Forward propagation, Back propagation y Gradiente descendente).

#Actividad 1
El problema que queremos resolver con esta red neuronal es si un vino es de muy bueno, regular o malo dependiendo sus caracteristicas

El link al dataset de Kagle es: https://www.kaggle.com/datasets/joebeachcapital/wine-quality

In [ ]:
#Importamos todas las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate
import networkx as nx
from random import random as rand
import itertools

In [ ]:
# Importa el archivo CSV 'winequality-red.csv'
# Usa el delimitador ';' ya que por el formato del archivo tiraba error
db_wine = pd.read_csv('./winequality-red.csv', delimiter=';')
db_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [ ]:
# Creamos las columnas de categorías para la clasificación
db_wine['bueno'] = db_wine['quality'].apply(lambda x: 1 if x >= 7 else 0)
db_wine['regular'] = db_wine['quality'].apply(lambda x: 1 if 5 <= x <= 6 else 0)
db_wine['malo'] = db_wine['quality'].apply(lambda x: 1 if x <= 4 else 0)

In [ ]:
# Separamos las características (X) y etiquetas (y)
X = db_wine.drop(columns=['quality', 'bueno', 'regular', 'malo'])
y = db_wine[['bueno', 'regular', 'malo']]

In [ ]:
#Hacemos las funciones necesarias para el SNN
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x) )

def sigmoid_derivative(x):
    return x * (1 - x)

In [ ]:
#Ponemos los pesos
training_inputs = np.array(X)
training_outputs = np.array(y)

In [ ]:
# Combinamos las training_inputs y las training_outputs
training_values = np.concatenate((training_inputs.T, training_outputs.T)).T
table = lambda arr : display(HTML(tabulate.tabulate(arr, tablefmt='html')))
table(training_values)

7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,0,1,0
7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,0,1,0
7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,0,1,0
11.2,0.28,0.56,1.9,0.075,17,60,0.998,3.16,0.58,9.8,0,1,0
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,0,1,0
7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,0,1,0
7.9,0.6,0.06,1.6,0.069,15,59,0.9964,3.3,0.46,9.4,0,1,0
7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10,1,0,0
7.8,0.58,0.02,2,0.073,9,18,0.9968,3.36,0.57,9.5,1,0,0
7.5,0.5,0.36,6.1,0.071,17,102,0.9978,3.35,0.8,10.5,0,1,0
6.7,0.58,0.08,1.8,0.097,15,65,0.9959,3.28,0.54,9.2,0,1,0


In [ ]:
# Establece la semilla para el generador de números aleatorios en 1.
np.random.seed(1)

In [ ]:
# Muestra la cantidad de características en el conjunto de datos de entrada de entrenamiento
training_inputs.shape[1]

11

In [ ]:
# Muestra la cantidad de características en el conjunto de datos de salida de entrenamiento
training_outputs.shape[1]

3

In [ ]:
# Generamos los weights
synaptic_weights = np.random.random((training_inputs.shape[1], 3))

synaptic_weights

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04],
       [3.02332573e-01, 1.46755891e-01, 9.23385948e-02],
       [1.86260211e-01, 3.45560727e-01, 3.96767474e-01],
       [5.38816734e-01, 4.19194514e-01, 6.85219500e-01],
       [2.04452250e-01, 8.78117436e-01, 2.73875932e-02],
       [6.70467510e-01, 4.17304802e-01, 5.58689828e-01],
       [1.40386939e-01, 1.98101489e-01, 8.00744569e-01],
       [9.68261576e-01, 3.13424178e-01, 6.92322616e-01],
       [8.76389152e-01, 8.94606664e-01, 8.50442114e-02],
       [3.90547832e-02, 1.69830420e-01, 8.78142503e-01],
       [9.83468338e-02, 4.21107625e-01, 9.57889530e-01]])

#Entrenamiento
Ahora vamos a pasar con la parte del entrenamiento del modelo

In [ ]:
#Creamos una función con la sigmoid para que nos devuelva una salida
forward = lambda x, w: sigmoid(np.dot(x, w))

In [ ]:
input_layer = training_inputs

# Genera un output haciendo la suma de los valores de input multiplicados por su peso sinaptico, y aplicandole la función sigmoide para normalizar a -1, 1
outputs = forward(input_layer, synaptic_weights)

outputs

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [ ]:
# Calculamos el error
error = training_outputs - outputs
error

array([[-1.00000000e+00,  1.22137855e-11, -1.00000000e+00],
       [-1.00000000e+00,  0.00000000e+00, -1.00000000e+00],
       [-1.00000000e+00,  2.77555756e-14, -1.00000000e+00],
       ...,
       [-1.00000000e+00,  1.99840144e-15, -1.00000000e+00],
       [-1.00000000e+00,  4.44089210e-16, -1.00000000e+00],
       [-1.00000000e+00,  9.19264664e-14, -1.00000000e+00]])

In [ ]:
# Esto nos sirve para ajustar los pesos en función del error
sigmoid_derivative(outputs)

array([[4.72127226e-10, 1.22137855e-11, 0.00000000e+00],
       [2.22044605e-16, 0.00000000e+00, 0.00000000e+00],
       [1.54254387e-12, 2.77555756e-14, 0.00000000e+00],
       ...,
       [1.33226763e-15, 1.99840144e-15, 0.00000000e+00],
       [2.22044605e-16, 4.44089210e-16, 0.00000000e+00],
       [9.87210313e-13, 9.19264664e-14, 0.00000000e+00]])

In [ ]:
# Multiplicamos el error por lo que hicimos anteriormente, para reducir el error
adjustments = error * sigmoid_derivative(outputs)
adjustments

array([[-4.72127226e-10,  1.49176557e-22, -0.00000000e+00],
       [-2.22044605e-16,  0.00000000e+00, -0.00000000e+00],
       [-1.54254387e-12,  7.70371978e-28, -0.00000000e+00],
       ...,
       [-1.33226763e-15,  3.99360833e-30, -0.00000000e+00],
       [-2.22044605e-16,  1.97215226e-31, -0.00000000e+00],
       [-9.87210313e-13,  8.45047523e-27, -0.00000000e+00]])

In [ ]:
# Realizamos el ajuste necesario para cada peso de la red, basandonos en el error de cada observación y su entrada correspondiente.
# Este resultado lo vamos a usar para actualizar los synaptic_weights
np.dot(input_layer.T, adjustments)

array([[-1.03664178e-03, -3.45803578e-06, -2.14029652e-07],
       [-7.48658727e-05, -2.82529930e-07, -9.69375909e-09],
       [-2.51700785e-05, -8.60281147e-08, -7.74757080e-09],
       [-2.63204534e-04, -9.92928512e-07, -3.37796306e-08],
       [-1.06734859e-05, -3.84470645e-08, -1.53473626e-09],
       [-5.34207384e-04, -2.00846303e-06, -6.80963575e-08],
       [-1.56996031e-03, -4.91056933e-06, -1.54835597e-07],
       [-1.34782703e-04, -4.79231920e-07, -2.07493491e-08],
       [-4.51699345e-04, -1.63914451e-06, -6.65748957e-08],
       [-7.90653232e-05, -2.79822542e-07, -1.25184091e-08],
       [-1.42070636e-03, -5.26740927e-06, -2.17231855e-07]])

In [ ]:
# Normalizamos los datos de entrada
training_inputs = (training_inputs - np.min(training_inputs)) / (np.max(training_inputs) - np.min(training_inputs))

epochs = 10000
learning_rate = 0.01

for iteration in range(epochs):

    # Arranca con los valores de training
    input_layer = training_inputs

    # Genera un output haciendo la suma de los valores de input multiplicados por su peso sinaptico, y aplicandole la función sigmoide para normalizar a -1, 1
    outputs = forward(input_layer, synaptic_weights)

    # Calculo el error
    error = training_outputs - outputs

    # Ajusto los pesos en funcion al error y al resultado obtenido
    adjustments = error * sigmoid_derivative(outputs)
    synaptic_weights += learning_rate * np.dot(input_layer.T, adjustments)

In [ ]:
print('Synaptic weights after training')
print(synaptic_weights)

print('Outputs after training: ')
print(outputs)

Synaptic weights after training
[[-6.34530900e+00  6.80477177e+01 -1.29764840e+01]
 [-1.68287195e+00  5.22287585e+00 -2.55023414e-01]
 [ 9.22743069e-01  2.98742463e+00 -2.20873107e-01]
 [-1.04203456e-01  2.10623503e+01 -2.21480895e+00]
 [ 2.44815725e-02  1.64953270e+00 -7.95219426e-02]
 [ 3.53810006e+00  1.75334928e+02 -9.05910297e+00]
 [-1.30658841e+01  9.76074400e+02 -2.35503072e+01]
 [-4.57957346e-01  8.34022699e+00 -6.47227788e-01]
 [-4.18545214e+00  2.68421861e+01 -4.19764935e+00]
 [ 3.39173154e-02  5.07986610e+00 -8.82724961e-02]
 [-6.56661450e+00  7.55330932e+01 -1.37593529e+01]]
Outputs after training: 
[[1.37565195e-01 1.00000000e+00 1.86368858e-02]
 [4.02753883e-02 1.00000000e+00 8.00211990e-04]
 [6.26487606e-02 1.00000000e+00 3.15531365e-03]
 ...
 [1.30551823e-01 1.00000000e+00 6.39711217e-03]
 [1.17519157e-01 1.00000000e+00 4.45433266e-03]
 [1.07900569e-01 1.00000000e+00 7.69263793e-03]]


In [ ]:
table(training_values)

7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,0,1,0
7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,0,1,0
7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,0,1,0
11.2,0.28,0.56,1.9,0.075,17,60,0.998,3.16,0.58,9.8,0,1,0
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,0,1,0
7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,0,1,0
7.9,0.6,0.06,1.6,0.069,15,59,0.9964,3.3,0.46,9.4,0,1,0
7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10,1,0,0
7.8,0.58,0.02,2,0.073,9,18,0.9968,3.36,0.57,9.5,1,0,0
7.5,0.5,0.36,6.1,0.071,17,102,0.9978,3.35,0.8,10.5,0,1,0
6.7,0.58,0.08,1.8,0.097,15,65,0.9959,3.28,0.54,9.2,0,1,0


In [ ]:
large = lambda x : display(HTML('<p style="font-size: 50px; color: #66D148">'+str(x)+'</p>'))

In [ ]:
a = 7.4 #@param {type:"number"}
b = 0.8 #@param {type:"number"}
c = 0.4 #@param {type:"number"}
d = 2.6 #@param {type:"number"}
e = 0.07 #@param {type:"number"}
f = 25 #@param {type:"number"}
g = 34 #@param {type:"number"}
h = 0.9 #@param {type:"number"}
i = 3.3 #@param {type:"number"}
j = 0.5 #@param {type:"number"}

input_layer = np.array([a, b, c, d, e, f, g, h, i, j])
res = sigmoid(np.dot(input_layer.shape[0], synaptic_weights.T))[0]
display(res)

thresh = 0.01
out = 0 if (res < thresh).any() else 1

large(out)

array([2.77123378e-28, 4.91337565e-08, 9.99901704e-01, 2.60757615e-01,
       5.60900066e-01, 1.00000000e+00, 1.80130166e-57, 1.01550875e-02,
       6.64985726e-19, 5.83989658e-01, 3.03078813e-29])